# Supervised Learning on IMDB Reviews

Learning using BERT base uncased over the original labels of the IMDB dataset

## Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('white')
sns.set_context('notebook')
%matplotlib inline

## Loading Dataset

In [2]:
train_df = pd.read_json('./train.json')
temp_df = pd.read_json('./valid.json')

In [3]:
valid_df = temp_df.iloc[:20000, :].reset_index(drop=True)
test_df = temp_df.iloc[20000:, :].reset_index(drop=True)

In [4]:
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_pandas(train_df, split='train')
valid_ds = Dataset.from_pandas(valid_df, split='valid')
test_ds = Dataset.from_pandas(test_df, split='test')

ds = DatasetDict({'train': train_ds, 'valid': valid_ds, 'test': test_ds})

In [5]:
ds['train'] = ds['train'].remove_columns(['__index_level_0__'])

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'weak_labels'],
        num_rows: 26544
    })
    valid: Dataset({
        features: ['text', 'label', 'weak_labels'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label', 'weak_labels'],
        num_rows: 8710
    })
})

## Tokenization

In [7]:
from transformers import AutoTokenizer

model_ckpt = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [8]:
def tokenize_ds(field):
    '''Tokenize examples from dataset.'''
    return tokenizer(field['text'], padding=True, truncation=True)

In [9]:
ds_encoded = ds.map(tokenize_ds, batched=True)

Map:   0%|          | 0/26544 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8710 [00:00<?, ? examples/s]

In [10]:
ds_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'weak_labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 26544
    })
    valid: Dataset({
        features: ['text', 'label', 'weak_labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label', 'weak_labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8710
    })
})

## Modeling

In [11]:
from sklearn.metrics import accuracy_score, f1_score

def compute_scores(preds):
    '''Compute scores of transformers predictions.'''
    logits, labels = preds
    pred = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, pred)
    f1 = f1_score(labels, pred)
    return {'Accuracy': acc, 'F1 Score': f1}

In [12]:
id2label = {0: 'negative', 1: 'positive'}
label2id = {'negative': 0, 'positive': 1}

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments

In [14]:
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=2, 
                          id2label=id2label, label2id=label2id))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
training_args = TrainingArguments(output_dir="./bert-base-uncased-imdb", 
                                  learning_rate=2e-5, 
                                  per_device_train_batch_size=1, 
                                  per_device_eval_batch_size=1, 
                                  num_train_epochs=3, 
                                  weight_decay=0.01, 
                                  evaluation_strategy="epoch", 
                                  save_strategy="epoch", 
                                  load_best_model_at_end=True, 
                                  logging_steps=500, 
                                  log_level="error", 
                                  push_to_hub=False)

In [16]:
from transformers import Trainer

trainer = Trainer(model=model, 
                  args=training_args, 
                  train_dataset=ds_encoded['train'], 
                  eval_dataset=ds_encoded['valid'], 
                  tokenizer=tokenizer, 
                  compute_metrics=compute_scores)

In [17]:
trainer.train()

C:\Users\ehhho\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 